# 🔗 유불리 판단 + 유사 조항 검색 + Ollama 기반 LLM 설명 생성

### 환경 테스트 코드(GPU 사용할 때)

In [1]:
# 설치 완료 후 실행할 검증 코드
import torch
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

print("=== 환경 검증 ===")
print(f"Python: {__import__('sys').version}")
print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

# GPU 테스트
if torch.cuda.is_available():
    device = torch.device("cuda")
    x = torch.rand(5, 3).to(device)
    print(f"GPU 테스트 성공: {x.device}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    print("GPU 사용 불가 - CPU 모드")

# Sentence Transformers 테스트
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    test_text = ["안녕하세요", "테스트 문장입니다"]
    embeddings = model.encode(test_text)
    print(f"✅ Sentence Transformers 정상 작동: {embeddings.shape}")
except Exception as e:
    print(f"❌ Sentence Transformers 오류: {e}")

print("=== 설치 완료 ===")


=== 환경 검증 ===
Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
NumPy: 1.24.4
PyTorch: 2.1.0+cu118
CUDA 사용 가능: True
GPU 테스트 성공: cuda:0
GPU 이름: GeForce GTX 1050 Ti


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\jhg-pc-02\Documents\TextMining\TextMining\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jhg-pc-02\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Sentence Transformers 정상 작동: (2, 384)
=== 설치 완료 ===


### 실행 코드

In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama


In [3]:
# 유불리 판단 모델
classification_dir = "../model/classification"
classifier_model = AutoModelForSequenceClassification.from_pretrained(classification_dir)
classifier_tokenizer = AutoTokenizer.from_pretrained(classification_dir)

# 의미 임베딩 모델
semantic_dir = "../model/legal-kr-sbert-contrastive"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
semantic_model = SentenceTransformer(semantic_dir).to(device)

dataset = torch.load(
    "embedding_dataset.pt",
    map_location=lambda storage, loc: storage.cuda() if torch.cuda.is_available() else storage
)

texts = dataset["texts"]
embeddings = dataset["embeddings"].to(device)

print(f"Device: {device}")
print(f"Embeddings device: {embeddings.device}")


Device: cuda
Embeddings device: cuda:0


In [4]:
def predict_unfairness(clauses):
    inputs = classifier_tokenizer(clauses, padding=True, truncation=True, return_tensors="pt")
    outputs = classifier_model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    return preds.tolist()  # 0: 유리, 1: 불리

In [5]:
import pandas as pd

# labeled.csv 읽기
df = pd.read_csv("labeled.csv")  # 또는 정확한 경로

In [6]:
def get_similar_clauses(query, top_k=5):
    query_emb = semantic_model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_emb, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    results = []
    for idx in top_results.indices:
        clause = texts[idx]
        # labeled.csv에서 basis 찾기
        matched = df[df["text"] == clause]
        basis = matched["basis"].values[0] if not matched.empty else ""
        results.append({"clause": clause, "basis": basis})
    
    return results


In [7]:
# Ollama LLM 설정
llm = ChatOllama(model="anpigon/EEVE-Korean-10.8B:latest")

# 프롬프트 템플릿
prompt_template = PromptTemplate(
    input_variables=["clause", "similar"],
    template="""다음은 서비스 약관의 조항입니다:

조항:
{clause}

유사한 조항들:
{similar}

이 조항이 왜 불리한지 설명해 주세요.
"""
)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# 설명 생성 함수
def generate_explanation(clause, similar_clauses):
    similar_text = "\n\n".join(
        [f"- 조항:\n{item['clause']}\n설명:\n{item.get('basis', '')}" for item in similar_clauses]
    )

    return llm_chain.run({
        "clause": clause,
        "similar": similar_text
    })

C:\Users\jhg-pc-02\AppData\Local\Temp\ipykernel_18420\2084197299.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="anpigon/EEVE-Korean-10.8B:latest")
C:\Users\jhg-pc-02\AppData\Local\Temp\ipykernel_18420\2084197299.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)


In [8]:
print("📥 여러 약관 조항을 한 줄씩 입력하세요. (빈 줄 입력 시 종료)\n")
input_clauses = []
while True:
    line = input("조항: ")
    if line.strip() == "":
        break
    input_clauses.append(line)

labels = predict_unfairness(input_clauses)

for clause, label in zip(input_clauses, labels):
    if label == 1:
        similar = get_similar_clauses(clause, top_k=5)
        explanation = generate_explanation(clause, similar)

        print(f"\n🔹 조항: {clause}")
        print("📎 유사 조항:")
        for i, s in enumerate(similar, 1):
            print(f"{i}. {s}")
        print(f"\n🧠 설명:\n{explanation}")

    else:
        print(f"\n🔹 조항: {clause}")
        print("✅ 유리한 조항으로 판단됨")

📥 여러 약관 조항을 한 줄씩 입력하세요. (빈 줄 입력 시 종료)



C:\Users\jhg-pc-02\AppData\Local\Temp\ipykernel_18420\2084197299.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run({



🔹 조항: 제10조(포인트 정정, 취소, 소멸)\n제5항 본 조 제3항에도 불구하고 본 약관 제7조 제1항 및 제2항에 따라 카드 탈퇴 및 자격상실된회원은 포인트가 소멸되고, 이에 대하여 회원은 어떠한 권리 주장할 수 없습니다. \n000의 경우, 기존 포인트 약관상  카드 해지 와  탈회 를 별도로 구분하지 아니하고,  탈퇴 라는 포괄적인 용어를 사용하였음.
📎 유사 조항:
1. {'clause': '(000 회원약관)\n제10조(포인트 정정, 취소, 소멸)\n제5항 본 조 제3항에도 불구하고 본 약관 제7조 제1항 및 제2항에 따라 카드 탈퇴 및 자격상실된 회원은 포인트가 소멸되고, 이에 대하여 회원은 어떠한 권리 주장할 수 없습니다.', 'basis': '특정 카드를 해지한 고객은 유효한 다른 카드를 이용하여 여전히 잔여포인트를 정상적으로 사용할 수 있는바, 해지 카드의 잔여포인트의 경우 해지 전 포인트와 그 취급을 달리할 특별한 이유가 존재하지 아니한다.'}
2. {'clause': '제7조 (포인트의 소멸)1항\n포인트는 다음 각 호에서 정한 유효기한 내에서 사용할 수 있으며 유효기간이 경과된 포인트는 월단위로 자동 소멸됩니다. 제7조 1항 1호\n00000의 유효기한은 적립일로부터 5년입니다. 제7조 2항\n제1항에도 불구하고 회원이 다음 각 호의 1에 해당하는 경우 적립된 포인트는 자동으로 소멸됩니다. 다만, 제1호 내지 제3호의 경우에는 그로 인하여 적립된 포인트만을 대상으로 합니다. 제7조2항 1호\n입회신청서의 허위기재, 사위 등의 방법으로 카드가 부정 발급된 경우 제7조2항 2호\n물품구매를 위장한 현금융통 등 불건전한 용도로 카드를 이용한 경우 제7조2항 3호\n도난, 위,변조 등의 방법으로 카드를 이용한 경우 제7조2항 4호\n법률 또는 회원의 요청으로 회원의 정보가 삭제되는 경우 제7조2항 5호\n회원이 사망하거나 카드 이용계약이 해지, 해제, 취소 및 기타의 사유로 카드 이용계약이 종료된 경우', 'basis': '탈회 또는 개인정